In [3]:
import os

path = "."
file_list = os.listdir(path)

import pandas as pd
import numpy as np
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.multioutput import MultiOutputRegressor
import optuna
from sklearn.model_selection import cross_val_score, train_test_split, cross_val_predict
from sklearn.metrics import mean_absolute_error
import shap
from lightgbm import LGBMRegressor, LGBMClassifier

from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
import warnings ; warnings.filterwarnings('ignore')
import time
from sklearn.metrics import f1_score, roc_auc_score, classification_report


In [12]:
train=pd.read_csv(path + '/2_Preprocessing/train_nanfill.csv', index_col='id')
test=pd.read_csv(path + '/2_Preprocessing/test_nanfill.csv', index_col='id')
submission=pd.read_csv(path + '/2_Preprocessing/submission_nanfill.csv', index_col='id')


train_cen = train-train.mean(axis=0)
train_cov = np.dot(train_cen.T, train_cen) / 59

print(train_cov)

w, v = np.linalg.eig(train_cov)

print('eigenvalue :', w)
print('eigenvector :\n', v)

print('explained variance ratio :', w/w.sum())


[[2.13019599e-17 1.50897985e-17 8.15931221e-18 ... 9.74346843e-19
  1.86008241e-18 2.46611195e-18]
 [1.50897985e-17 1.73419897e-17 8.75622747e-18 ... 8.84573511e-19
  1.65601403e-18 2.26273182e-18]
 [8.15931221e-18 8.75622747e-18 4.54111371e-18 ... 6.64285023e-19
  1.28841361e-18 1.91237323e-18]
 ...
 [9.74346843e-19 8.84573511e-19 6.64285023e-19 ... 1.44125722e-17
  2.52469881e-17 3.95385323e-17]
 [1.86008241e-18 1.65601403e-18 1.28841361e-18 ... 2.52469881e-17
  5.19690291e-17 8.69943753e-17]
 [2.46611195e-18 2.26273182e-18 1.91237323e-18 ... 3.95385323e-17
  8.69943753e-17 1.57818683e-16]]
eigenvalue : [3.24306968e-15 3.14527936e-16 1.64630482e-16 6.36645983e-17
 3.50183013e-17 1.75558078e-17 1.43945788e-17 8.74895677e-18
 6.27680651e-18 5.55455107e-18 3.29398774e-18 2.69442388e-18
 1.45314516e-18 1.14340143e-18 7.61080209e-19 7.16504932e-19
 5.43728561e-19 4.72048265e-19 4.28959700e-19 3.16870328e-19
 2.99829838e-19 2.83511272e-19 1.57365671e-20 1.93279797e-20
 3.71679438e-20 5.912

In [9]:
U, D, V_t = np.linalg.svd(train_cen)

print('singular value :', D)
print('singular vector :\n', V_t.T)
print('explained variance ratio :', D**2 / np.sum(D**2))


singular value : [4.37425549e-07 1.36224624e-07 9.85555601e-08 6.12879376e-08
 4.54541503e-08 3.21837328e-08 2.91424115e-08 2.27197810e-08
 1.92440013e-08 1.81029974e-08 1.39407775e-08 1.26083706e-08
 9.25935012e-09 8.21344533e-09 6.70102472e-09 6.50182982e-09
 5.66391959e-09 5.27739023e-09 5.03076757e-09 4.32381190e-09
 4.20594346e-09 4.08988570e-09 3.70849978e-09 3.46664051e-09
 3.38340654e-09 3.32436685e-09 3.16442525e-09 2.96723387e-09
 2.70681291e-09 2.54075565e-09 2.11614290e-09 1.86777050e-09
 1.48084729e-09 1.06787209e-09 9.63564975e-10]
singular vector :
 [[ 0.01297687  0.03691462 -0.09106438 ... -0.02711551 -0.02837519
  -0.00423761]
 [ 0.0165748   0.05163538 -0.10276958 ...  0.22222598  0.17611638
  -0.1155142 ]
 [ 0.00985272  0.02473212 -0.05396271 ... -0.4240363  -0.42238969
   0.49133144]
 ...
 [ 0.02691625 -0.13384841 -0.13423194 ...  0.00490656 -0.00449774
   0.00484891]
 [ 0.05177179 -0.28262052 -0.30001444 ...  0.00827535  0.00730042
   0.00230893]
 [ 0.08036771 -0.49

In [15]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)
pca.fit(train)
print('singular value :', pca.singular_values_)
print('singular vector :\n', pca.components_.T)



singular value : [4.37425549e-07 1.36224624e-07 9.85555601e-08]
singular vector :
 [[ 0.01297687  0.03691462 -0.09106438]
 [ 0.0165748   0.05163538 -0.10276958]
 [ 0.00985272  0.02473212 -0.05396271]
 [ 0.00862499  0.01253866 -0.02985919]
 [ 0.00991337  0.01386223 -0.03357322]
 [ 0.01360552  0.01883931 -0.04484207]
 [ 0.0157091   0.0242359  -0.06979001]
 [ 0.01501728  0.01936878 -0.06777049]
 [ 0.01524292  0.01659257 -0.06624959]
 [ 0.01900162  0.02980725 -0.08778559]
 [ 0.03493113  0.0639746  -0.12960871]
 [ 0.05810091  0.09761395 -0.15990435]
 [ 0.11546446  0.14763437 -0.17315744]
 [ 0.2044943   0.2188974  -0.23365734]
 [ 0.30969693  0.26223458 -0.23161679]
 [ 0.39115134  0.24803302 -0.17152448]
 [ 0.40639394  0.16219903 -0.049627  ]
 [ 0.36589757  0.0532116   0.06372746]
 [ 0.32191778 -0.03347752  0.15439452]
 [ 0.28332666 -0.10888773  0.23587008]
 [ 0.2639302  -0.17772143  0.283428  ]
 [ 0.24227543 -0.2454717   0.28466211]
 [ 0.19093671 -0.26457231  0.19964726]
 [ 0.12073621 -0.204

In [16]:
print('eigen_value :', pca.explained_variance_)
print('explained variance ratio :', pca.explained_variance_ratio_)




eigen_value : [1.91360247e-17 1.85590041e-18 9.71416984e-19]
explained variance ratio : [0.83424578 0.08090902 0.04234947]
